<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget file.weasoft.com/images.zip
!wget file.weasoft.com/summary.csv

In [ ]:
!unzip images.zip

In [ ]:
!wget file.weasoft.com/29_4010.pt

--2024-01-24 00:42:59--  http://file.weasoft.com/29_4010.pt
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149608395 (143M) [application/vnd.snesdev-page-table]
Saving to: ‘29_4010.pt’

29_4010.pt          100%[===================>] 142.68M  18.9MB/s    in 8.7s    

2024-01-24 00:43:08 (16.4 MB/s) - ‘29_4010.pt’ saved [149608395/149608395]



In [ ]:
!git clone https://github.com/suanfaxiaohuo/SwinOCSR.git
!pip install focal_loss_torch wandb
!pip3 install deepsmiles yacs tqdm

In [ ]:
import sys
import os
import argparse
from tqdm import tqdm
import deepsmiles
from typing import Any, cast, Callable, List, Tuple, Union
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

In [ ]:
import sys
sys.path.append("./SwinOCSR/model/Swin-transformer-focalloss")
sys.path.append("./SwinOCSR/model/")
from pre_transformer import Transformer
class FocalLossModelInference:
    """
    Inference Class
    """
    def __init__(self):
        # Load dictionary that maps tokens to integers
        word_map_path = './SwinOCSR/Data/500wan/500wan_shuffle_DeepSMILES_word_map'
        self.word_map = torch.load(word_map_path)
        self.inv_word_map = {v: k for k, v in self.word_map.items()}

        # Define device, load models and weights
        self.dev = "cuda" if torch.cuda.is_available() else "cpu"
        # self.args, config = self.get_inference_config()
        # self.encoder = build_model(config, tag=False)
        self.decoder = self.build_decoder()
        # self.load_checkpoint("./swin_transform_focalloss.pth")
        self.decoder = self.decoder.to(self.dev).eval()
        # self.encoder = self.encoder.to(self.dev).eval()

    def load_checkpoint(self, checkpoint_path):
        """
        Load checkpoint and update encoder and decoder accordingly

        Args:
            checkpoint_path (str): path of checkpoint file
        """
        print(f"=====> {checkpoint_path} <=====")
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        # encoder_msg = self.encoder.load_state_dict(checkpoint['encoder'],
        #                                            strict=False)
        decoder_msg = self.decoder.load_state_dict(checkpoint['decoder'],
                                                   strict=False)
        # print(f"Encoder: {encoder_msg}")
        print(f"Decoder: {decoder_msg}")
        del checkpoint
        torch.cuda.empty_cache()

    def build_decoder(self):
        """
        This method builds the Transformer decoder and returns it
        """
        self.decoder_dim = 256  # dimension of decoder RNN
        self.ff_dim = 2048
        self.num_head = 8
        self.dropout = 0.1
        self.encoder_num_layer = 6
        self.decoder_num_layer = 6
        self.max_len = 277
        self.decoder_lr = 5e-4
        self.best_acc = 0.
        return Transformer(dim=self.decoder_dim,
                           ff_dim=self.ff_dim,
                           num_head=self.num_head,
                           encoder_num_layer=self.encoder_num_layer,
                           decoder_num_layer=self.decoder_num_layer,
                           vocab_size=len(self.word_map),
                           max_len=self.max_len,
                           drop_rate=self.dropout,
                           tag=False)
transformer_ = FocalLossModelInference()

In [ ]:
import base64
import pandas as pd
import os
# df = pd.read_csv("images_rows.csv")
# os.makedirs("images",exist_ok=1)
# def save_img(id):
#   row = df[df["id"]==id]
#   print(row["image"])
#   with open(f"images/{id}.png", "wb") as fh:
#       fh.write(base64.decodebytes(bytes(row["image"].item().replace("data:image/png;base64,",""), "utf-8")))
# df["id"].apply(save_img)

In [ ]:
import sys
sys.path.append("./SwinOCSR/model/Swin-transformer-focalloss")
sys.path.append("./SwinOCSR/model/")

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
swin = torchvision.models.swin_t()
swin.head = swin.avgpool = swin.flatten = torch.nn.Identity()
mynet = swin
class ImageEncoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.eff = mynet.to(device)
    self.projection = torch.nn.Linear(768,256).to(device)
  def forward(self, images):
    features = self.eff(images)
    features = torch.flatten(features, start_dim=2, end_dim=3)
    features = torch.permute(features, (0, 2, 1))
    return self.projection(features)
class Image2SMILES(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, image, text_in, xmask):
    image_feature = self.encoder(image)
    out = self.decoder(text_in, image_feature, x_mask=xmask)
    return out

In [ ]:
mod = torch.load("29_4010.pt", map_location=device) #it is model not dict

In [ ]:
def pad_pack(sequences):
    maxlen = max(map(len, sequences))
    batch = torch.LongTensor(len(sequences),maxlen).fill_(0)
    for i,x in enumerate(sequences):
        batch[i,:len(x)] = torch.LongTensor(x)
    return batch, maxlen

In [ ]:
class SMILESGenerator(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, image, text_in_, max_len, beam): #just changed beam=1 and it runs so beam cannot be random number???  yeah it has to been factor of 676, which is 2 2 13 13 but original paper did not use beam search
    image_feature = self.encoder(image)
    top_n = list([list(i) for i in torch.tensor(text_in_).repeat(beam,1).detach().cpu().numpy()])
    # print(top_n)
    image_feature = image_feature.repeat_interleave(beam, dim=0)#.repeat_interleave(beam)
    for i in range(max_len):
      if beam == 1:
        padded_text, l = pad_pack(top_n)
        padded_text = padded_text.to(device)
        out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
        out = out[0,-1,:]
        next = torch.argmax(out)
        top_n[0] += [next.detach().cpu().item()]
        if next == 78:
          return top_n
        continue
      padded_text, l = pad_pack(top_n)
      padded_text = padded_text.to(device)
      out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
      out = out[:,-1,:]
      next = torch.topk(torch.flatten(out), beam)
      indices2d = np.array(np.unravel_index(next.indices.cpu().numpy(), out.shape)).T
      new_top_n = []
      count = 0
      for j in indices2d:
        beam_number = j[0]
        char = j[1]
        if char == 78:
          count += 1
          new_top_n.append(top_n[beam_number])
        else:
          new_top_n.append(top_n[beam_number] + [char])
      if count == beam:
        return new_top_n
      top_n = new_top_n
    return top_n

In [ ]:
gen = SMILESGenerator(mod.encoder, mod.decoder)

In [ ]:
def triangle_mask(size):
    mask = 1- np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    mask = torch.autograd.Variable(torch.from_numpy(mask))
    return mask

In [ ]:
!wget file.weasoft.com/reverse.map

--2024-01-24 00:43:45--  http://file.weasoft.com/reverse.map
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 875
Saving to: ‘reverse.map’

reverse.map         100%[===================>]     875  --.-KB/s    in 0s      

2024-01-24 00:43:45 (125 MB/s) - ‘reverse.map’ saved [875/875]



In [ ]:
reversed_word_map = {}
import json
with open("reverse.map","r") as f:
  reversed_word_map = json.loads(f.read())

In [ ]:
from focal_loss.focal_loss import FocalLoss
m = torch.nn.Softmax(dim=-1)
lf = FocalLoss(gamma=2, ignore_index=0)#torch.nn.CrossEntropyLoss(label_smoothing=0.1, reduction="none")
def loss_fn(pred, truth):
  pred = m(pred)
  l = lf(pred, truth)
  return l

In [ ]:
def mask_acc(pred, truth):
    pred = torch.argmax(pred, -1)
    mask = truth != 0
    match_case = truth == pred
    return torch.sum(mask*match_case)/torch.sum(mask)

In [ ]:
BATCH_SIZE = 1

In [64]:
import pandas as pd
molecules_rows = pd.read_csv("summary.csv")

In [ ]:
print("\n".join(list(set([i.split("_")[0] for i in os.listdir("images")]))))

In [86]:
import random
converter = deepsmiles.Converter(rings=True, branches=True)
ids = molecules_rows["cid"]
def getitem(names, train=True):

  ti = []
  to = []
  imgs = []
  for i in range(16):
    name = random.choice(names)
    # if (train and name.startswith("11203"))or (not train and not name.startswith("11203")):
    # if train == name.startswith("11203"):
    #   continue

    img = Image.open(f"images/{name}").convert("RGB")
    if train:
      img = img.rotate(random.choice([0,90,270]), expand=0).resize((400,400))
      img.crop((random.random()*20,random.random()*20,400-random.random()*20,400-random.random()*20))
    img = np.array(img.resize((400,400)), dtype="float32")
    if train:
      img[:,:,0] *= random.random()*0.2+0.9
      img[:,:,1] *= random.random()*0.2+0.9
      img[:,:,2] *= random.random()*0.2+0.9
    img = torch.tensor(np.array(img)).permute(2, 0, 1).to(torch.float32).to(device)
    if train:
      img*=torch.round(torch.tanh(5*torch.rand((3,400,400)))).to(device)
    # print(name)
    # print(converte//r.encode(molecules_rows[molecules_rows["cid"]==int(name.split("_")[0])]["canonicalsmiles"].item()))
    smiles = [transformer_.word_map[i] for i in converter.encode(molecules_rows[molecules_rows["cid"]==int(name.split("_")[0])]["canonicalsmiles"].item())]
    ti.append([77] + smiles)
    to.append(smiles + [78])
    imgs.append(img)
  return torch.stack(imgs), ti, to


In [ ]:
torch.stack([torch.tensor([1,2]) for i in range(10)])

In [ ]:
import pylab
pylab.imshow(getitem(os.listdir("images")[0])[0].cpu().permute(0, 2, 3, 1).numpy()[0]/255)

In [ ]:
molecules_rows

In [ ]:
# print("\n".join([str(i) for i in df["cid"].unique()]))

In [74]:
for i in mod.decoder.parameters():
  i.require_grad = True

In [82]:
def val():
  running_loss = 0
  mod.train(False)
  ids = [i for i in os.listdir("images") if i.startswith("11203")]
  # ids = [i if i.startswith("11203") else None for i in os.listdir("images")]
  valacc = []
  np.random.shuffle(ids)
  for i in ids:
    if i==None:
      continue
    start_index = i
    image, text_in, text_out = getitem(ids, train=False)

    image = image.to(device)
    text_out = pad_pack(text_out)[0].to(device)
    padded_x = pad_pack(text_in)

    xmask = triangle_mask(padded_x[1]).to(device)
    text_in = padded_x[0].to(device)

    outputs = mod(image, text_in, xmask)
    loss = loss_fn(outputs, text_out)



    running_loss += loss.item()
    valacc.append(mask_acc(outputs.detach(), text_out).item())
  mod.train(True)
  return running_loss/len(ids), np.mean(valacc)


In [88]:
mod = torch.load("29_4010.pt", map_location=device) #it is model not dict

In [90]:
losses = []
access = []
val_acc = []
optimizer = torch.optim.AdamW(
   mod.parameters(),
   lr=0.00005)

import pylab
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
running_loss = 0
for epoch in range(8):
  mod.train(True)
  ids = [i for i in os.listdir("images") if not i.startswith("11203")]
  np.random.shuffle(ids)
  for i in range(len(ids)//16):
    if i==None:
      continue
    start_index = i
    image, text_in, text_out = getitem(ids, start_index)

    image = image.to(device) #mutli process cannot use cuda so moved here
    # image = torch.permute(image, (0, 3, 1, 2))
    text_out = pad_pack(text_out)[0].to(device)
    padded_x = pad_pack(text_in)

    xmask = triangle_mask(padded_x[1]).to(device)
    text_in = padded_x[0].to(device)

    optimizer.zero_grad()
    outputs = mod(image, text_in, xmask)
    #loss = loss_fn(outputs, text_outi) guaibude yyixiazinamegao
    loss = loss_fn(outputs, text_out)

    loss.backward()

    optimizer.step()

    running_loss += loss.item()
    losses.append(loss.item())
    access.append(mask_acc(outputs.detach(), text_out))

  val_acc.append(val()[1])
  scheduler.step()
  pylab.plot([i.item() for i in access])
  pylab.show()
  pylab.clf()
  print(optimizer.param_groups[0]["lr"])

KeyboardInterrupt: 

In [ ]:
running_loss = 0
for epoch in range(8):
  mod.train(True)
  ids = [i if not i.startswith("11203") else None for i in os.listdir("images")]
  np.random.shuffle(ids)
  for i in ids:
    if i==None:
      continue
    start_index = i
    image, text_in, text_out = getitem(start_index)

    image = image.to(device) #mutli process cannot use cuda so moved here
    # image = torch.permute(image, (0, 3, 1, 2))
    text_out = pad_pack(text_out)[0].to(device)
    padded_x = pad_pack(text_in)

    xmask = triangle_mask(padded_x[1]).to(device)
    text_in = padded_x[0].to(device)

    optimizer.zero_grad()
    outputs = mod(image, text_in, xmask)
    #loss = loss_fn(outputs, text_outi) guaibude yyixiazinamegao
    loss = loss_fn(outputs, text_out)

    loss.backward()

    optimizer.step()

    running_loss += loss.item()
    losses.append(loss.item())
    access.append(mask_acc(outputs.detach(), text_out))

  val_acc.append(val()[1])
  scheduler.step()
  pylab.plot([i.item() for i in access])
  pylab.show()
  pylab.clf()
  print(optimizer.param_groups[0]["lr"])

In [ ]:
res = np.array([i.item() for i in access[2500:]])

In [ ]:
np.sum(res==1)/len(res)

In [ ]:
pylab.plot([i.item() for i in access])
pylab.plot(np.arange(len(val_acc))*(len(access)/len(val_acc)),val_acc)